In [5]:
import pandas as pd

df_trips = pd.read_parquet('green_tripdata_2025-11.parquet')
df_zones = pd.read_csv('taxi_zone_lookup.csv')

# df_trips['lpep_pickup_datetime'] = pd.to_datetime(df_trips['lpep_pickup_datetime'])

In [6]:
# Question 3.For the trips in November 2025, how many trips had a trip_distance of less than or equal to 1 mile?

november_trips = df_trips[
    (df_trips['lpep_pickup_datetime'] >= '2025-11-01') &
    (df_trips['lpep_pickup_datetime'] < '2025-12-01')
    ]
short_trips = november_trips[november_trips['trip_distance'] <= 1.0].shape[0]

print(f"Number of trips with distance <= 1 mile: {short_trips}")

Number of trips with distance <= 1 mile: 8007


In [8]:
# Question 4. Which was the pick up day with the longest trip distance? Only consider trips with trip_distance less than 100 miles

df_filtered = df_trips[df_trips['trip_distance'] < 100]

longest_trip = df_filtered.loc[df_filtered['trip_distance'].idxmax()]

result_date = longest_trip['lpep_pickup_datetime'].date()
result_distance = longest_trip['trip_distance']

print(f"The day with the longest trip distance was: {result_date}")
print(f"The trip distance was: {result_distance} miles")

The day with the longest trip distance was: 2025-11-14
The trip distance was: 88.03 miles


In [9]:
# Question 5. Which was the pickup zone with the largest total_amount (sum of all trips) on November 18th, 2025?

nov_18_trips = df_trips[
    df_trips['lpep_pickup_datetime'].dt.date == pd.to_datetime('2025-11-18').date()
    ]

zone_totals = nov_18_trips.groupby('PULocationID')['total_amount'].sum().reset_index()

result = pd.merge(
    zone_totals,
    df_zones[['LocationID', 'Zone']],
    left_on='PULocationID',
    right_on='LocationID'
)

max_zone = result.sort_values('total_amount', ascending=False).iloc[0]

print(f"Pickup zone with largest total amount: {max_zone['Zone']}")
print(f"Total amount: ${max_zone['total_amount']:.2f}")

Pickup zone with largest total amount: East Harlem North
Total amount: $9281.92


In [10]:
# Question 6. For the passengers picked up in the zone named "East Harlem North" in November 2025, which was the drop off zone that had the largest tip?
november_trips = df_trips[
    (df_trips['lpep_pickup_datetime'].dt.year == 2025) &
    (df_trips['lpep_pickup_datetime'].dt.month == 11)
    ]


trips_with_zones = november_trips.merge(
    df_zones[['LocationID', 'Zone']],
    left_on='PULocationID',
    right_on='LocationID'
).rename(columns={'Zone': 'pickup_zone'}).merge(
    df_zones[['LocationID', 'Zone']],
    left_on='DOLocationID',
    right_on='LocationID',
    suffixes=('_pickup', '_dropoff')
).rename(columns={'Zone': 'dropoff_zone'})

east_harlem_trips = trips_with_zones[
    trips_with_zones['pickup_zone'] == 'East Harlem North'
    ]

max_tip_trip = east_harlem_trips.loc[east_harlem_trips['tip_amount'].idxmax()]

print("\nResults:")
print(f"Pickup Zone: East Harlem North")
print(f"Dropoff Zone with largest tip: {max_tip_trip['dropoff_zone']}")
print(f"Tip amount: ${max_tip_trip['tip_amount']:.2f}")


Results:
Pickup Zone: East Harlem North
Dropoff Zone with largest tip: Yorkville West
Tip amount: $81.89
